In [233]:
import pandas as pd
import os
import re
import datetime

In [234]:
file_names = list()
for path, subdirs, files in os.walk('./Csv/'):
    for filename in files:
        file_names.append(filename)

In [235]:
Health_By_Risks = []
for file_name in file_names:
    pattern = r'(Health By Risk2|Health By Risk3)'
    if re.search(pattern=pattern,string=file_name):
        Health_By_Risks.append(file_name)
Health_By_Risks = sorted(Health_By_Risks)

In [236]:
dict_df = dict()
date_list = []
date = ''
for file_name in Health_By_Risks:
    if file_name.split('-')[0].strip() != date:
        date = file_name.split('-')[0].strip()
        date_list.append(date)
        dict_df[date] = list()
    df = pd.read_csv('./Csv/{}'.format(file_name), index_col=0, header=0)
    dict_df[date].append(df)

In [237]:
col_names = ['Risk',
            'Pulls',
            '%Pulls',
           'Total Pulls',
           'Average Deads',
           'Total Deads',
           'Case Fatality',
           'Month']

all_df_month = []
for date in date_list:
    temp_df = dict_df[date][0]  #[1] mean second page pdf, month xxxx
#     temp_df.columns = col_name
    temp_df = temp_df.iloc[2:]
    temp_df.reset_index(drop=True, inplace=True)
    temp_df = temp_df.iloc[:6]
    temp_df['Month'] = [date for i in range(6)]
    all_df_month.append(temp_df)

df_month = pd.concat(all_df_month, axis=0, ignore_index=True)
df_month.columns = col_names
df_month['%Deads']=None
df_month[['Average Deads', '%Deads']] = df_month['Average Deads'].apply(lambda x: pd.Series(x.split(' ')))
df_month = df_month[['Month',
               'Risk',
            'Pulls',
            '%Pulls',
           'Total Pulls',
           'Average Deads',
               '%Deads',
           'Total Deads',
           'Case Fatality']]
#### function start ####
def convert(x):
    x = str(x)
    x = x.strip().replace(',','')
    if re.search(pattern = r'[a-zA-Z]+',string=x):
        return x
    elif re.search(pattern=r'%', string=x):
        return float(x.replace('%',''))/100
    elif x=='-':
        return 0
    elif re.search(pattern=r'$', string=x):
        return float(x.replace('$',''))
    else:
        return float(x)
#### function ends ####


for col in df_month.columns:
    df_month[col] = df_month[col].apply(convert)
df_month['Month'] = df_month['Month'].apply(lambda x: datetime.datetime.strptime(x, '%B %Y'))
df_month = df_month[df_month['Risk']!='Respiratory']   #if not need the average and total of month

In [238]:
all_df_ytd = []
for date in date_list:
    temp_df = dict_df[date][1]  #[1] mean second page pdf, month xxxx
#     temp_df.columns = col_name
    temp_df = temp_df.iloc[2:]
    temp_df.reset_index(drop=True, inplace=True)
    temp_df = temp_df.iloc[:6]
    temp_df['Month'] = [date for i in range(6)]
    all_df_ytd.append(temp_df)

df_ytd = pd.concat(all_df_ytd, axis=0, ignore_index=True)
df_ytd.columns = col_names
df_ytd['%Deads']=None
df_ytd[['Average Deads', '%Deads']] = df_ytd['Average Deads'].apply(lambda x: pd.Series(x.split(' ')))
df_ytd = df_ytd[['Month',
               'Risk',
            'Pulls',
            '%Pulls',
           'Total Pulls',
           'Average Deads',
               '%Deads',
           'Total Deads',
           'Case Fatality']]


for col in df_ytd.columns:
    df_ytd[col] = df_ytd[col].apply(convert)
df_ytd['Month'] = df_ytd['Month'].apply(lambda x: datetime.datetime.strptime(x, '%B %Y'))
df_ytd = df_ytd[df_ytd['Risk']!='Respiratory']   #if not need the average and total of month

In [239]:
df_month.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 70 entries, 1 to 83
Data columns (total 9 columns):
Month            70 non-null datetime64[ns]
Risk             70 non-null object
Pulls            70 non-null float64
%Pulls           70 non-null float64
Total Pulls      70 non-null float64
Average Deads    70 non-null float64
%Deads           70 non-null float64
Total Deads      70 non-null float64
Case Fatality    70 non-null float64
dtypes: datetime64[ns](1), float64(7), object(1)
memory usage: 5.5+ KB


In [240]:
df_ytd.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 70 entries, 1 to 83
Data columns (total 9 columns):
Month            70 non-null datetime64[ns]
Risk             70 non-null object
Pulls            70 non-null float64
%Pulls           70 non-null float64
Total Pulls      70 non-null float64
Average Deads    70 non-null float64
%Deads           70 non-null float64
Total Deads      70 non-null float64
Case Fatality    70 non-null float64
dtypes: datetime64[ns](1), float64(7), object(1)
memory usage: 5.5+ KB


In [241]:
df_month.head()

,Month,Risk,Pulls,%Pulls,Total Pulls,Average Deads,%Deads,Total Deads,Case Fatality
1,2017-04-01,Low Risk,400.0,0.0154,4395.0,34.0,0.0012,378.0,0.0860
2,2017-04-01,Moderate Risk,381.0,0.0178,12963.0,30.0,0.0014,1003.0,0.0774
3,2017-04-01,High Risk,411.0,0.0153,6568.0,40.0,0.0014,639.0,0.0973
4,2017-04-01,Holstein,245.0,0.0127,245.0,14.0,0.0007,14.0,0.0571
5,2017-04-01,Natural,139.0,0.0088,277.0,10.0,0.0007,20.0,0.0722


In [242]:
df_ytd.head()

,Month,Risk,Pulls,%Pulls,Total Pulls,Average Deads,%Deads,Total Deads,Case Fatality
1,2017-04-01,Low Risk,2171.0,0.0965,23885.0,148.0,0.0053,1625.0,0.0680
2,2017-04-01,Moderate Risk,2270.0,0.1228,77166.0,146.0,0.0074,4956.0,0.0642
3,2017-04-01,High Risk,3786.0,0.1596,60575.0,285.0,0.0124,4559.0,0.0753
4,2017-04-01,Holstein,2766.0,0.1433,2766.0,85.0,0.0044,85.0,0.0307
5,2017-04-01,Natural,2242.0,0.1444,4483.0,96.0,0.0063,191.0,0.0426
